In [66]:
import pandas as pd
import os
from datetime import datetime, timedelta
import time
import ast
from collections import Counter


In [42]:
# 'ContentsList.xlsx' 파일 불러오기
file_path1 = r'C:\Users\ch.kang\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\ContentsList.xlsx'
df1 = pd.read_excel(file_path1)
df1.columns

Index(['20240119', 'contents_url', 'category', 'board1', 'board2', 'title',
       'post_start', 'post_end'],
      dtype='object')

In [50]:
def process_excel_file(excel_file_path, df1):
    # 엑셀 파일을 데이터프레임으로 읽어오기
    df = pd.read_excel(excel_file_path)

    # Mapping category to corresponding URL prefix
    category_url_mapping = {
        'blog_msp': 'https://blog.naver.com/mysecondplace',
        'blog_hanpro911': 'https://blog.naver.com/hanpro911'
    }

    # Applying the mapping to create the 'composed_url' column
    df['composed_url'] = df.apply(lambda row: f'{category_url_mapping.get(row["category"], "")}/{int(row["Unnamed: 0"])}', axis=1)

    # Merging df with df1 on the composed_url column
    df_result = pd.merge(df, df1, left_on='composed_url', right_on='contents_url', how='left')

    # id_header
    id_header = df_result.columns[8]

    # selected_columns
    df_result = df_result[[id_header, 'contents_url', 'date_info', 'views', 'acq_path', 'acq_path_detail', 'composed_url']]
    
    # Convert "date_info" to string and remove the decimal part
    df_result['date_info'] = df_result['date_info'].astype(str).str.split('.').str[0]
    # Convert "date_info" to datetime with the specified format and then to 8-digit integer
    df_result['date_info'] = pd.to_datetime(df_result['date_info'], format='%Y%m%d', errors='coerce').dt.strftime('%Y%m%d').astype(int)

    return df_result

In [51]:
# Directory containing the Excel files
directory_path = r'C:\Users\ch.kang\OneDrive - 클리 주식회사\문서 - 클리주식회사\003_Project-3\07_콘텐츠성과측정\DailyRawdata'

# List of Excel files in the directory starting with "blog"
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx') and file.startswith('blog')]

# 데이터를 저장할 딕셔너리
blog_data_dict = {}

# 각 엑셀 파일을 읽어서 데이터를 딕셔너리에 저장
for excel_file in excel_files:
    # 엑셀 파일의 경로
    excel_file_path = os.path.join(directory_path, excel_file)

    # Process the Excel file using the function
    processed_df = process_excel_file(excel_file_path, df1)

    # 데이터프레임을 딕셔너리에 추가, key는 파일 이름으로 부여
    blog_data_dict[excel_file] = processed_df

# 데이터프레임 딕셔너리 출력
for key, value in blog_data_dict.items():
    print(f"DataFrame {key}:\n{value}\n")

DataFrame blog_20240115.xlsx:
   20240119                                       contents_url  date_info  \
0   BM-0001  https://blog.naver.com/mysecondplace/223203815685   20240115   
1   BM-0002  https://blog.naver.com/mysecondplace/223208249149   20240115   
2   BM-0003  https://blog.naver.com/mysecondplace/223209462852   20240115   
3   BM-0004  https://blog.naver.com/mysecondplace/223212353603   20240115   
4   BM-0005  https://blog.naver.com/mysecondplace/223215558503   20240115   
5   BM-0006  https://blog.naver.com/mysecondplace/223229049237   20240115   
6   BM-0007  https://blog.naver.com/mysecondplace/223240007276   20240115   
7   BM-0008  https://blog.naver.com/mysecondplace/223241084754   20240115   
8   BM-0009  https://blog.naver.com/mysecondplace/223247154141   20240115   
9   BM-0010  https://blog.naver.com/mysecondplace/223250496328   20240115   
10      NaN                                                NaN   20240115   
11  BM-0011  https://blog.naver.com/mysecondpl

In [56]:
# Assuming blog_data_dict is your dictionary
target_value = 'BM-0001'

# Initialize lists to store views and acq_path for the target_value
views_values = []
acq_path_values = []
acq_path_detail_values = []

# Iterate through the dictionary
for key, df in blog_data_dict.items():
    # Check if target_value is in the '20240119' column
    if target_value in df.iloc[:, 0].values:
        # Get the views and acq_path for the rows with the target_value
        views_values.extend(df.loc[df.iloc[:, 0] == target_value, 'views'].tolist())
        acq_path_values.extend(df.loc[df.iloc[:, 0] == target_value, 'acq_path'].tolist())
        acq_path_detail_values.extend(df.loc[df.iloc[:, 0] == target_value, 'acq_path_detail'].tolist())

# Print the results
print(f"Views for {target_value}: {views_values}")
print(f"Acquisition Paths for {target_value}: {acq_path_values}")


Views for BM-0001: [0, 3, 3, 0]
Acquisition Paths for BM-0001: ['[]', "['네이버 블로그_PC(100%, 3.0건)']", "['www.notion.so(100%, 3.0건)']", '[]']


In [61]:
# blog_data_dict is dictionary
df1_values = df1.iloc[:, 0].values  # Assuming df1 is a DataFrame

# Initialize lists to store views and acq_path for each target_value
all_views_values = []
all_acq_path_values = []
all_acq_path_detail_values = []

# Iterate through values in df1
for target_value in df1_values:
    # Initialize lists to store views and acq_path for the current target_value
    views_values = []
    acq_path_values = []
    acq_path_detail_values = []

    # Iterate through the dictionary
    for key, df in blog_data_dict.items():
        # Check if target_value is in the first column
        if target_value in df.iloc[:, 0].values:
            # Get the views and acq_path for the rows with the target_value
            views_values.extend(df.loc[df.iloc[:, 0] == target_value, 'views'].tolist())
            acq_path_values.extend(df.loc[df.iloc[:, 0] == target_value, 'acq_path'].tolist())
            acq_path_detail_values.extend(df.loc[df.iloc[:, 0] == target_value, 'acq_path_detail'].tolist())

    # Append the results for the current target_value to the overall lists
    all_views_values.append(views_values)
    all_acq_path_values.append(acq_path_values)
    all_acq_path_detail_values.append(acq_path_detail_values)

# Create a DataFrame from the results
result_df = pd.DataFrame({
    'contents_id': df1_values,
    'views': all_views_values,
    'acquisition_paths': all_acq_path_values,
    'acquisition_paths_detail': all_acq_path_detail_values
})


In [65]:
result_df

,contents_id,views,acquisition_paths,acquisition_paths_detail
0,BH-0001,"[0, 0]","[[], []]","[[], []]"
1,BH-0002,"[3, 4, 1, 0]","[['네이버 뷰검색_모바일(50%, 1.5건)', '네이버 통합검색_모바일(50%,...","[['공주시 시골집매매(50%, 1.5건)', '농가주택 구매(50%, 1.5건)'..."
2,BH-0003,"[0, 4, 3, 2]","[[], ['네이버 블로그_PC(50%, 2.0건)', '네이버 블로그_PC(50%...","[[], ['https://blog.naver.com/PostList.naver?b..."
3,BH-0004,"[1, 1, 5, 3]","[['네이버 통합검색_모바일(100%, 1.0건)'], ['네이버 블로그_PC(10...","[['로우테크(100%, 1.0건)'], ['https://blog.naver.co..."
4,BH-0005,"[0, 1, 3, 2]","[[], ['네이버 블로그_PC(100%, 1.0건)'], ['네이버 블로그_모바일...","[[], ['https://blog.naver.com/PostView.naver?b..."
...,...,...,...,...
104,HH-0053,[],[],[]
105,HH-0054,[],[],[]
106,HH-0055,[],[],[]
107,HH-0056,[],[],[]


In [81]:
result_df.at[1, 'acquisition_paths_detail']

["['공주시 시골집매매(50%, 1.5건)', '농가주택 구매(50%, 1.5건)']",
 "['촌집 구매시 조심(25%, 1.0건)', '충천도시골집매매(25%, 1.0건)', '입장 농가주택 빈집(25%, 1.0건)', '촌집 구매(25%, 1.0건)']",
 "['시골주택 구매(100%, 1.0건)']",
 '[]']

In [82]:
# Function to handle variations of empty lists
def convert_to_list(x):
    try:
        # Try to convert to a list
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        # Handle variations of empty lists
        if x in ('[]', "['[]', '[]']"):
            return []
        else:
            # If unable to convert, return as is
            return x


In [83]:
# Assuming result_df is your existing DataFrame
result_df['acquisition_paths'] = result_df['acquisition_paths'].apply(convert_to_list)


In [90]:
# Function to extract counts and keywords from acquisition_paths
def extract_naver_info(paths):
    search_naver_count = 0
    search_keywords = []

    for path in paths:
        if path.startswith("['네이버"):
            # Extract counts and remove non-numeric characters
            counts = [float(''.join(filter(str.isdigit, x.split('%')[0]))) for x in path.split(',')]
            search_naver_count += sum(counts)

            # Extract keywords starting with '네이버'
            keywords = [x.split(',')[0][2:].strip() for x in path.split("', '")]
            search_keywords.extend(keywords)

    return search_naver_count, search_keywords

# Apply the function to create new columns
result_df[['search_naver', 'search_keyword']] = result_df.apply(lambda row: pd.Series(extract_naver_info(row['acquisition_paths_detail'])), axis=1)

# Display the resulting DataFrame
result_df[['contents_id', 'search_naver', 'search_keyword']]

,contents_id,search_naver,search_keyword
0,BH-0001,0,[]
1,BH-0002,0,[]
2,BH-0003,0,[]
3,BH-0004,0,[]
4,BH-0005,0,[]
...,...,...,...
104,HH-0053,0,[]
105,HH-0054,0,[]
106,HH-0055,0,[]
107,HH-0056,0,[]


In [87]:
# Apply the function to create new columns
result_df[['search_naver', 'search_keyword']] = result_df.apply(lambda row: pd.Series(extract_naver_info(row['acquisition_paths'])), axis=1)

# Display the resulting DataFrame
result_df[['contents_id', 'search_naver', 'search_keyword']]

ValueError: could not convert string to float: " 1.5건)'"

In [72]:
flat_acquisition_paths

['[]',
 '[]',
 "['네이버 뷰검색_모바일(50%, 1.5건)', '네이버 통합검색_모바일(50%, 1.5건)']",
 "['네이버 뷰검색_모바일(50%, 2.0건)', '네이버 뷰검색_모바일(50%, 2.0건)', '네이버 통합검색_모바일(50%, 2.0건)', '네이버 통합검색_모바일(50%, 2.0건)']",
 "['네이버 통합검색_PC(100%, 1.0건)']",
 '[]',
 '[]',
 "['네이버 블로그_PC(50%, 2.0건)', '네이버 블로그_PC(50%, 2.0건)', 'Bing(25%, 1.0건)', '네이버 통합검색_모바일(25%, 1.0건)']",
 "['네이버 통합검색_모바일(100%, 3.0건)']",
 "['네이버 이미지검색_PC(50%, 1.0건)', '네이버 통합검색_모바일(50%, 1.0건)']",
 "['네이버 통합검색_모바일(100%, 1.0건)']",
 "['네이버 블로그_PC(100%, 1.0건)']",
 "['네이버 블로그_모바일(25%, 1.25건)', '네이버 이미지검색_PC(25%, 1.25건)', '네이버 이미지검색_모바일(25%, 1.25건)', '네이버 통합검색_모바일(25%, 1.25건)']",
 "['네이버 이미지검색_PC(33.33%, 0.9998999999999999건)', '네이버 이미지검색_모바일(33.33%, 0.9998999999999999건)', '네이버 통합검색_PC(33.33%, 0.9998999999999999건)']",
 '[]',
 "['네이버 블로그_PC(100%, 1.0건)']",
 "['네이버 블로그_모바일(100%, 3.0건)']",
 "['네이버 블로그_PC(100%, 2.0건)', '네이버 블로그_PC(100%, 2.0건)']",
 '[]',
 "['네이버 블로그_PC(100%, 1.0건)']",
 "['네이버 블로그_모바일(100%, 4.0건)']",
 "['네이버 블로그_PC(100%, 1.0건)']",
 '[]',
 '[]',
 "['네이버 블로그_PC(

# header information
header_info =  [
    {
      "header": "contents_id",
      "name": "콘텐츠 구분자",
      "description": "콘텐츠를 식별하는 고유 번호나 코드"
    },
    {
      "header": "page_url",
      "name": "페이지 주소",
      "description": "콘텐츠가 노출되는 웹 페이지의 URL"
    },
    {
      "header": "category",
      "name": "구분",
      "description": "페이지의 종류나 카테고리로 네이버 블로그(hanpro911), 네이버 블로그(MSP), 네이버 카페(MSP), 유튜브 등."
    },
    {
      "header": "board1",
      "name": "게시판1",
      "description": "페이지가 속한 최상위 게시판 또는 섹션, 재생목록."
    },
    {
      "header": "board2",
      "name": "게시판2",
      "description": "페이지가 속한 하위 게시판 또는 섹션, 재생목록."
    },
    {
      "header": "title(query)",
      "name": "제목",
      "description": "페이지의 제목으로 html head>title 값을 파싱"
    },
    {
      "header": "detail",
      "name": "상세설명",
      "description": "페이지 내용에 대한 상세 추가 설명, 서식 없음"
    },
    {
      "header": "start_date",
      "name": "게재시작일",
      "description": "페이지 게재가 시작된 날짜"
    },
    {
      "header": "end_date",
      "name": "게재종료일",
      "description": "페이지 게재가 종료된 날짜"
    },
    {
      "header": "total_views",
      "name": "총조회수",
      "description": "페이지의 총 조회 횟수"
    },
    {
      "header": "top_inflow_channel",
      "name": "최대 유입채널",
      "description": "페이지로 가장 많이 유입된 채널"
    },
    {
      "header": "top_inflow_count",
      "name": "최대 유입채널_수",
      "description": "가장 많이 유입된 채널의 유입 횟수"
    },
    {
      "header": "previous_page1_url",
      "name": "직전페이지1_url",
      "description": "첫 번째로 많이 방문한 직전 페이지의 URL"
    },
    {
      "header": "previous_page1_count",
      "name": "직전페이지1_수",
      "description": "첫 번째 직전 페이지의 방문 횟수"
    },
    {
      "header": "previous_page1_ratio",
      "name": "직전페이지1_비율",
      "description": "총 조회수 대비 첫 번째 직전 페이지의 방문 횟수"
    },
    {
      "header": "previous_page2_url",
      "name": "직전페이지2_url",
      "description": "두 번째로 많이 방문한 직전 페이지의 URL"
    },
    {
      "header": "previous_page2_count",
      "name": "직전페이지2_수",
      "description": "두 번째 직전 페이지의 방문 횟수"
    },
    {
      "header": "previous_page2_ratio",
      "name": "직전페이지2_비율",
      "description": "총 조회수 대비 두 번째 직전 페이지의 방문 횟수"
    },
    {
      "header": "previous_page3_url",
      "name": "직전페이지3_url",
      "description": "세 번째로 많이 방문한 직전 페이지의 URL"
    },
    {
      "header": "previous_page3_count",
      "name": "직전페이지3_수",
      "description": "세 번째 직전 페이지의 방문 횟수"
    },
    {
      "header": "previous_page3_ratio",
      "name": "직전페이지3_비율",
      "description": "총 조회수 대비 세 번째 직전 페이지의 방문 횟수"
    },
    {
      "header": "landing_visits",
      "name": "랜딩수",
      "description": "랜딩 페이지(pseudo id가 최초로 방문한 페이지)에 대한 총 조회 횟수"
    },
    {
      "header": "average_stay_time",
      "name": "평균체류시간",
      "description": "페이지에서의 평균 체류 시간"
    },
    {
      "header": "average_scroll_count",
      "name": "평균스크롤횟수",
      "description": "페이지에서의 평균 스크롤 횟수"
    },
    {
      "header": "average_click_count",
      "name": "평균클릭횟수",
      "description": "페이지에서의 평균 클릭 횟수"
    },
    {
      "header": "conversion1_event",
      "name": "전환1의 이름",
      "description": "전환1으로 측정된 이벤트의 이름"
    },
    {
      "header": "conversion1_count",
      "name": "전환1의 수",
      "description": "전환1으로 측정된 이벤트의 총합"
    },
    {
      "header": "conversion1_rate",
      "name": "전환1의 전환율",
      "description": "전환1으로 측정된 이벤트의 총합/페이지의 총 조회수"
    },
    {
      "header": "conversion2_event",
      "name": "전환2의 이름",
      "description": "전환2으로 측정된 이벤트의 이름"
    },
    {
      "header": "conversion2_count",
      "name": "전환2의 수",
      "description": "전환2으로 측정된 이벤트의 총합"
    },
    {
      "header": "conversion2_rate",
      "name": "전환2의 전환율",
      "description": "전환2으로 측정된 이벤트의 총합/페이지의 총 조회수"
    },
    {
      "header": "next_page1_url",
      "name": "직후페이지1_url",
      "description": "첫 번째로 많이 방문한 직후 페이지의 URL"
    },
    {
      "header": "next_page1_count",
      "name": "직후페이지1_수",
      "description": "첫 번째 직후 페이지의 방문 횟수"
    },
    {
      "header": "next_page2_url",
      "name": "직후페이지2_url",
      "description": "두 번째로 많이 방문한 직후 페이지의 URL"
    },
    {
      "header": "next_page2_count",
      "name": "직후페이지2_수",
      "description": "두 번째 직후 페이지의 방문 횟수"
    },
    {
      "header": "next_page3_url",
      "name": "직후페이지3_url",
      "description": "세 번째로 많이 방문한 직후 페이지의 URL"
    },
    {
      "header": "next_page3_count",
      "name": "직후페이지3_수",
      "description": "세 번째 직후 페이지의 방문 횟수"
    }
]